In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
#from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL-Advanced


# 데이터 전달하기

In [5]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

runnable = RunnableParallel(
    # 전달된 입력을 그대로 반환하는 Runnable을 설정합니다.
    passed=RunnablePassthrough(),
    # 입력의 "num" 값에 3을 곱한 결과를 반환하는 Runnable을 설정합니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 입력의 "num" 값에 1을 더한 결과를 반환하는 Runnable을 설정합니다.
    modified=lambda x: x["num"] + 1,
)

# {"num" : 1}을 입력으로 Runnable을 실행
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [6]:
r = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3)
r.invoke({"num": 1})

{'num': 1, 'mult': 3}

# 검색기 예제

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 텍스트로부터 FAISS 벡터 저장소를 생성
vectorstore = FAISS.from_texts(
    [
        "테디는 랭체인 주식회사에서 근무를 하였습니다.",
        "셜리는 테디와 같은 회사에서 근무하였습니다.",
        "테디의 직업은 개발자입니다.",
        "셜리의 직업은 디자이너입니다.",
    ],
    embedding=OpenAIEmbeddings(),
)

# 벡터 저장소를 검색기로 사용
retriever = vectorstore.as_retriever()

# 템플릿을 정의
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# 템플릿으로부터 채팅 프롬프트를 생성
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 모델을 초기화
model = ChatOpenAI(model="gpt-4o-mini")

# 문서를 포맷팅하는 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

# 검색 체인을 구성
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [8]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("테디의 직업은 무엇입니까?")

'테디의 직업은 개발자입니다.'

In [9]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("셜리의 직업은 무엇입니까?")

'셜리의 직업은 디자이너입니다.'